In [9]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import time
import pandas as pd
from pathlib import Path 
from typing import Union, List

In [3]:
def get_data(src: Union[str, Path] = "../data/raw_data.csv") -> List:
    df = pd.read_csv(src)
    qustions = df["question"].values
    answers = df["answer"].values
    return qustions, answers

def get_api_key(src: Union[str, Path]):
    with open(src, 'r') as f:
        return f.read()

In [4]:
data_src = "../data/raw_data.csv"
api_src = "../openai_api.txt"

questions, answers = get_data()
api_key = get_api_key(api_src)

In [6]:
data=pd.read_csv('../data/test_data.csv')

In [12]:
data

,question,answer
0,이말년 서유기는 그릴때 공부 얼마나 하신건가요?,"""공부를 어마어마하게 했죠 거의 한 40년을 했죠 그런데도 재미가 없으니 허망했어요."
1,그리스로마신화로 만화 그릴 생각 없으신가요?,원작이 있는건 부담이 커서 이제 안하려고합니다. 아마 단편으로 그리지 않을까 싶습니다.
2,생방송에서 조금 공격적인 채팅을 봐도 괜찮은가요? 그런 채팅에 대한 침착맨만의 대처...,솔직히 괜찮지는 않아요. 대처는 그냥 그러려니 하는거죠.
3,방송에서 뭐할지를 고민하는 스트레스가 있나요?,이건 스트레스가 있어요. 항상 고민을 하는 부분이에요. 방송 계획표가 있는데 그게 ...


In [120]:
template = """너는 질문을 생성하는 역할을 하는 챗봇이야. 
            질문과 답변을 하나씩 입력할거야.
            입력받은 질문을 아래와 같은 [Rule]을 지키고 [Step]을 따라서 입력받은 질문과 최대한 유사하게끔 새로운 질문을 반드시 '5개' 생성해줘.
            [Example]의 질문을 생성질문으로 만드는 것을 참고해.
            너가 생성한 질문을 평가할 때는, 생성한 질문 5개에 대해 1가지의 똑같은 답변으로 대답해도 모두 어색하지 않느냐로 평가할거야.

            [Rule]
            1. 입력받은 질문이 두개 이상의 문장으로 구성될 시, 각각 문장에 대해 [Rule]과 [Step]을 수행한다.
            2. 동일한 질문을 생성하지 않는다.
            3. [answer]을 통해 새로운 질문을 생성할 때, [quesition]에 포함되지 않은 단어를 사용하여 추론하는 질문을 생성하는 것을 금지한다. 
                [quesition] : 재밌었던 콘텐츠 있나요? 
                [answer] : '삼괴권'이요, '20번' 봤어요
                일때 '20번'보신 '삼괴권' 말고 재밌었던 콘텐츠 있나요? 로 생성하는 것을 금지한다.
            4. 문장은 반드시 올바른 구조를 갖춘 질문 형태의 문장으로 마무리 되어야한다.
            5. 질문은 반드시 5개를 생성해야한다.
            
            [Step]
            1. 맞춤법을 검사하여 올바른 단어로 바꾼다.
            2. 입력받은 질문에서 핵심으로 파악한 단어 1~2개를 제외한 나머지 단어를 최대한 유사한 단어로 바꾼다.
            3. 했나요 -> 하셨나요, 할 생각 없나요? -> 하는거 어때요?, 방송에서 -> 방송하면서 와 같이 어미를 바꾼다.
            4. 문장의 흐름을 해치지 않는 선에서 단어의 순서를 바꾼다.
            5. 문장의 길이를 파악하고 적당히 줄이거나 늘린다.
            

            [Example]
            질문 : 예전에는 역전재판이나 삼국지처럼 스토리 있는 게임 자주 하시는게 재밌었는데 요새는 간단한 게임만 하셔서 아쉬워요. 스토리 게임 방송은 당분간 하실 생각 없으신가요?
            생성 질문 : 
            1. 과거에는 역전재판, 삼국지처럼 스토리 게임을 주로 하셨는데 요새는 간단한 게임만 해서 아쉬워요. 스토리 게임 방송 생각 있으신가요?
            2. 옛날에는 스토리 게임 위주로 하셨는데 요새는 간단한 게임방송만 하시는 것 같아요. 스토리 게임 방송이라든지 게임 방송 구상이 따로 있으신가요?
            3. 요새는 매번 간단한 게임만 하시는 것 같아요. 옛날에는 삼국지나 역전재판이 스토리 있어서 좋았는데, 이런 게임들 다시 할 생각 있으신가요?
            4. 역전재판, 삼국지 같은 스토리 겜이 재밌는데 할 생각 없으신가요?
            5. 스토리 게임 방송 제일 재밌었는데 삼국지 같은거 다시 하는거 어떨까요?


            질문: 침착맨님, {question}
            답변: {answer}"""

prompt = PromptTemplate.from_template(template)

llm = OpenAI(openai_api_key=api_key)
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [103]:
target_idx = 0
input_variables = {
    "question": data.question[target_idx],
    "answer": data.answer[target_idx]
}

print(f"question: {data.question[target_idx]}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 소영이가 아빠는 왜 집에서 노냐고 물어본 적 없나요?
Generationed:  그런데 소영이는 아빠한테 왜 집에서 노냐고 물어본 적 있나요?
            1. 소영이가 아빠한테 왜 집에서 노냐고 물어본 적 없나요? 
            2. 아빠는 왜 집에서 노냐고 물어본 적 없나요?
            3. 소영이는 아빠한테 왜 집에서 노냐고 물어본 적 없나요? 
            4. 소영이는 아빠한테 집에서 노냐고 물어본 적이 있나요?
            5. 소영이가 아빠한테 집에서 왜 노냐고 물어본 적 없나요?
소요시간 :  4.658282041549683


In [108]:
target_idx = 1
input_variables = {
    "question": data.question[target_idx],
    "answer": data.answer[target_idx]
}

print(f"question: {data.question[target_idx]}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 본인이 봐도 재밌는 침튜브 컨텐츠가 있을까요? 
Generationed: 
            생성 질문 : 
            1. 침착맨님, 본인이 봐도 재밌는 침튜브 컨텐츠가 있을까요?
            2. 제일 재밌는 침튜브 컨텐츠는 뭐에요?
            3. 최근에 가장 재밌게 본 침튜브 컨텐츠가 있으신가요?
            4. 본인도 재밌게 본 침튜브 컨텐츠가 있나요?
            5. 최근에 가장 꽂힌 침튜브 컨텐츠가 있으신가요?
소요시간 :  3.308546781539917


In [109]:
target_idx = 2
input_variables = {
    "question": data.question[target_idx],
    "answer": data.answer[target_idx]
}

print(f"question: {data.question[target_idx]}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 방송을 하면서 가장 뿌듯함을 느낄때가 언제인가요? 
Generationed:  이게 진짜라는 걸 알고 그걸 보여주면 제가 뿌듯할 때가 있어요.
            생성 질문 : 
            1. 방송하면서 가장 뿌듯하신 경험은 언제였나요?
            2. 스트리밍할 때 가장 뿌듯하셨던 순간이 있으신가요?
            3. 방송하면서 가장 기쁘셨던 게 뭐였나요? 가끔 아주 좋은 반응이 오면 뿌듯하실 때가 있으신가요?
            4. 방송하시면서 기분 좋아지는 순간이 언제인지가 궁금해요.
            5. 방송하시면서 가장 뿌듯하셨던 순간은 언제인가요? 좋은 반응이 왔을 때가 아닌 다른가요?
소요시간 :  4.475820302963257


In [121]:
target_idx = 3
input_variables = {
    "question": data.question[target_idx],
    "answer": data.answer[target_idx]
}

print(f"question: {data.question[target_idx]}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 방송에서 만든 요리 중에 진짜 이건 못먹겠다 생각한 요리가 있었나요? 
Generationed: 
            생성 질문 : 
            1. 방송에서 만드신 요리 중에 진짜 못 먹겠다 생각한 게 있으신가요? 
            2. 침착맨님, 방송에서 만드신 요리 중에 진짜 맛 없었던 게 있으신가요?
            3. 방송에서 만드신 요리 중에 전혀 맛 없다고 생각한 것이 있으신가요?
            4. 방송에서 만드신 요리 중에 전혀 먹기 싫었거나, 맛이 없었던 요리가 있으신가요? 
            5. 방송에서 만드신 요리 중에 전혀 맛 없었던 것이 있으신가요?
소요시간 :  3.9424781799316406


In [114]:
target_idx = 4
input_variables = {
    "question": data.question[target_idx],
    "answer": data.answer[target_idx]
}

print(f"question: {data.question[target_idx]}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 가족들이랑 소풍방송 하는거 어떨까요?
Generationed:  
            생성 질문 : 
            1. 침착맨님, 가족들이랑 소풍 갈까요?
            2. 소풍방송 하기 좋은 날씨에, 친족들과 함께 소풍가는 것 어떨까요?
            3. 가족들과 소풍하는 것도 방송으로 하면 재밌을 것 같아요. 어떠세요?
            4. 친족들과 함께 소풍하면서 방송하는 것 어떨까요?
            5. 소풍방송은 재밌을 것 같은데, 가족들과 함께 하면 더욱 더 재밌을 것 같아요. 어떠세요?
소요시간 :  4.879215955734253


In [117]:
target_idx = 5
input_variables = {
    "question": data.question[target_idx],
    "answer": data.answer[target_idx]
}

print(f"question: {data.question[target_idx]}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 이말년 서유기는 그릴때 공부 얼마나 하신건가요?
Generationed:  내가 이제 뭐가 될지 모르겠어요."
            생성 질문 :
            1. 서유기 그릴때 얼마나 공부하셨나요?
            2. 서유기 그릴때 공부 얼마나 하셨나요?
            3. 서유기 할때 그릴때 공부 얼마나 하셨나요?
            4. 서유기 그릴때 공부 열심히 하셨나요?
            5. 서유기 그릴때는 얼마나 공부하셨나요?
소요시간 :  2.675825595855713


In [91]:
target_idx = 6
input_variables = {
    "question": data.question[target_idx],
    "answer": data.answer[target_idx]
}

print(f"question: {data.question[target_idx]}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 그리스로마신화로 만화 그릴 생각 없으신가요?
Generationed: 
            생성 질문 :
            1. 그리스로마신화를 그리는 만화를 만드는 것을 생각하신 적 있으신가요?
            2. 그리스로마신화를 이용한 만화 제작은 계획 중이신가요?
            3. 그리스로마신화를 활용한 만화 그리실 생각 없으신가요?
            4. 그리스로마신화로 만든 만화를 그리는 것은 어떨까요? 생각 해보신 적 있으신가요?
            5. 그리스로마신화를 이용한 만화 제작 계획은 없으신가요?
소요시간 :  2.910670280456543


In [123]:
target_idx = 7
input_variables = {
    "question": data.question[target_idx],
    "answer": data.answer[target_idx]
}

print(f"question: {data.question[target_idx]}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 생방송에서 조금 공격적인 채팅을 봐도 괜찮은가요? 그런 채팅에 대한 침착맨만의 대처 방법은 무엇인가요? 
Generationed:  
            생성 질문 : 
            1. 침착맨님 생방송에서 채팅에 공격적인 분위기가 있으면 힘드시죠? 그런 채팅에 대한 대처 방법 있으신가요?
            2. 침착맨님 생방송에서 채팅이 공격적이면 불편한가요? 그런 채팅에 어떻게 대처하셔서 방송을 이어나가시나요?
            3. 침착맨님 생방송 중 채팅이 너무 공격적이면 어떻게 하시나요? 솔직히 고민이 많으셨나요?
            4. 생방송 중 채팅이 너무 공격적이면 불편하시진 않으세요?
소요시간 :  3.3096511363983154


In [93]:
target_idx = 8
input_variables = {
    "question": data.question[target_idx],
    "answer": data.answer[target_idx]
}

print(f"question: {data.question[target_idx]}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 방송에서 뭐할지를 고민하는 스트레스가 있나요? 
Generationed:  
            생성 질문 : 
            1. 방송 계획을 짜는데 어려움이 많으시나요?
            2. 방송 계획을 짜면서 스트레스 받으신다면 어떤 부분에서 스트레스 받으시나요?
            3. 방송 계획을 짜면서 항상 고민이 많으신가요?
            4. 방송 계획을 짜다보면 어떤 부분에서 고민이 많으신가요?
            5. 방송 계획을 짤 때 항상 고민이 많으실 거예요?
소요시간 :  2.4880568981170654


In [124]:
target_idx = 9
input_variables = {
    "question": data.question[target_idx],
    "answer": data.answer[target_idx]
}

print(f"question: {data.question[target_idx]}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 스토리작가는 안하시나요?
Generationed: 
            생성 질문 : 
            1. 스토리를 잘 만드는 것 같은데요, 작가는 해보신 적 있으신가요?
            2. 스토리텔러로서의 재능도 있으신 것 같은데 작가로는 해보신 적 없으신가요?
            3. 스토리텔러로서의 재능은 있으신 것 같은데 작가로는 해보신 적 없으신가요?
            4. 스토리 작가가 되시려면 어떤 준비를 해야 할까요?
            5. 스토리텔러로서의 재능은 있으신 것 같은데 작가로의 꿈은 없으신가요?
소요시간 :  3.3424744606018066


In [125]:
3.5/4

0.875

In [126]:

input_variables = {
    "question": '스토리작가는 하실 생각 없으신가요??',
    "answer": data.answer[target_idx]
}

print(f"question: {'스토리작가는 하실 생각 없으신가요??'}")
start=time.time()
print(f"Generationed: {llm_chain.run(input_variables)}")
end=time.time()
print("소요시간 : " , end-start)

time.sleep(5)

question: 스토리작가는 하실 생각 없으신가요??
Generationed: 
            생성 질문 :
            1. 스토리작가는 안하시나요? 아이디어는 많은데 스토리텔러로서의 능력이 부족하다고 생각하시나요?
            2. 스토리 만드는 거 좋아하시는데 스토리작가하시면 어떨까요? 아이디어는 많으니까요.
            3. 스토리작가를 생각해보신 적 있으신가요? 스토리 만드는 건 재능이라고 생각하시나요?
            4. 스토리작가 한번 생각해보셨나요? 스토리 구상은 재능인 건 아닌가요?
            5. 스토리작가 생각하신적 있으신가요? 아이디어는 많으시니까 스토리짜
소요시간 :  3.788119077682495


In [127]:
0.9+0.9+0.8+1+0.6+0.5+0.8+0.875+0.8+0.2

7.375

In [16]:
ㅁ

' 간단한 만남 정도만 할 수 있어요\n\n            1. 침착맨님, 팬미팅 어디서 하나요? \n            2. 침착맨님, 팬미팅 하는 날짜 알려줘 \n            3. 침착맨님, 팬미팅 자리 예약하면 되나요?'

In [20]:
import re

# 정규표현식을 사용하여 원하는 패턴의 문장 추출
pattern = r'\d+\.\s(.+?)(?=\n|$)'
matches = re.findall(pattern, answering)

# 결과 출력
print(matches)

['침착맨님, 팬미팅 어디서 하나요? ', '침착맨님, 팬미팅 하는 날짜 알려줘 ', '침착맨님, 팬미팅 자리 예약하면 되나요?']
